In [1]:
import pandas as pd 
import warnings


warnings.filterwarnings('ignore')
#Los archivos están en la misma carpeta que este .ipynb
clientes = pd.read_excel('./maestro_clientes.xlsx')
productos = pd.read_csv('./maestro_productos.csv') 
proveedor_1 = pd.read_csv('./sellout-proveedor1.csv.gz',compression = 'gzip',sep = '|') # el separador de estos archivos
proveedor_2 = pd.read_csv('./sellout-proveedor2.csv.gz',compression = 'gzip',sep = '|') # es | 

In [2]:
# Regularmente empiezo viendo los detalles más simples para asegurarme que tengo la tabla correcta.
# clientes.info()
# clientes.sample(10)
# productos.info()
# productos.sample(10)
# proveedor_1.info()
# proveedor_1.sample(10)
# proveedor_2.info()
# proveedor_2.sample(10)

In [3]:
# Regularmente cambio las columnas a tipos adecuados en otras tablas.
precios_numericos = pd.to_numeric(proveedor_2['Price'].str.replace('$','',regex=True),errors='raise')
proveedor_2f = proveedor_2.assign(Price = precios_numericos)
proveedor_2f['Date'] = pd.to_datetime(proveedor_2['Date'],format = '%Y/%m/%d')
proveedor_1f = proveedor_1.copy()
proveedor_1f['Date'] = pd.to_datetime(proveedor_1['Date'],format= '%Y-%m-%d SO Diaria Piezas')
# proveedor_1f.info()
# proveedor_2f.info() # Para probar que los tipos están correctos y no fallaron en la conversión.
 
# Los siguientes puntos se hacen más fáciles si agrupamos estas dos tablas en una.
proveedor_1f = proveedor_1f.rename({'Units':'Pieces'},axis = 'columns')
data = pd.concat([proveedor_1f,proveedor_2f],ignore_index=True) # ignorar el indice es bueno para no tener indices repetidos
# data.info()

In [4]:
# Cuando revise los documentos note que hay algunos detalles en los datos:
# Hay algunas filas con precio 0 y piezas 0, esto probablemente es un error.
# Hay algunas filas con piezas mayor que 0 y precio 0, posiblemente un error.

ceros = data[data['Price']+data['Pieces'] == 0]
cero_costo = data[(data['Price']==0) & (data['Pieces'] > 0)]
# ceros.info(),cero_costo.info() # Hay 568,000 en el primer caso y 110 en el segundo.

# Creo que quitar estos datos es necesario, pero en cualquier caso se puede cambiar de tabla si es necesario.
data_limpia = data[~data.index.isin(list(cero_costo.index)+list(ceros.index))]
#data_limpia.info()

# Otro posible problema son entradas en la columna piezas  que no son enteros, lo cual es un posible error.
# Estos representan casi la mitad de todas las  entradas en la tabla de ventas.

print(str(len(data_limpia[data_limpia['Pieces'] != data_limpia['Pieces'].astype(int)]))+ ' son entradas con una fracción en "Pieces"')

# Opte por no considerarlo un error, pero sino fuera un test preguntaria un poco más por este detalle.
# data_entera = data_limpia[data_limpia[(data_limpia['Pieces'] != data_limpia['Pieces'].astype(int)])]
# En caso de que esas entradas se consideren un error se puede ocupar esta tabla.


2397549 son entradas con una fracción en "Pieces"


In [5]:
# Parece que la tabla de clientes se conecta a la de ventas por medio de 'Store_Id = NUM TDA'
# sin embargo existen entradas que su Store_Id no esta en ninguna entrada de clientes
sin_tienda = data[~data['Store_Id'].isin(clientes['NUM TDA'])]

# [174, 712, 681,  78,  94, 301, 675, 676, 709, 711]  son todos los Store_id que no están en 
# NUM TDA de un total de 2324 Store_Id distinto, no es necesario remover estas filas de  los datos pero no tendrán 
# un proveedor ni una tienda asignada, El total de filas en ese estado es 34100.

# Por otro lado  
sin_compras = clientes[~ clientes['NUM TDA'].isin(data['Store_Id'])]
# son todas las tiendas que no tienen ninguna venta (entrada en sellout) su tamaño es 133 de un total de 2448
# sin_compras.groupby(['CADENA','ESTADO']).count()['NUM TDA']
# muestra que 5 son del proveedor_1 y las demás del proveedor_2 (interesante pero en otro tema).

In [6]:
# Quiero saber cuantos estados distinto hay en la tabla 
print('Hay ' + str(len(clientes['ESTADO'].unique())) +' distintos en la tabla')

# El numero de diferentes estados en la tabla de clientes es 32 
# Quiero saber que en que estado esta cada tienda y a que cadena pertenece
estados = clientes[['CADENA','ESTADO','NUM TDA']].drop_duplicates()
estados_activos = estados[['CADENA','ESTADO']].drop_duplicates().groupby(['CADENA']).count()
#   |CADENA      |   ESTADO |
#   |:-----------|---------:|
#   | proveedor_1|       26 |
#   | proveedor_2|       32 |
# Nos dice que el proveedor 2 no tiene ninguna tienda en 6 estados y entonces técnicamente no cubren todo el país 
# Incluso suponiendo que el país no es México pues proveedor 2 tiene 32 estados en esa tabla.
# Estados.drop_duplicates().groupby(['CADENA','ESTADO']).count() muestra que la cantidad de tiendas entre el 
# proveedor 1 y proveedor 2 es grande, siendo proveedor 2 significativamente más grande.

# Quiero saber si todos los estados tienen al menos una venta con cada proveedor.
# Primero agrego los estados pegandolos con 'Store_ID' y 'NUM TDA'
# Hay 4 tiendas que están repetidas
clientes = clientes[~clientes['NUM TDA'].duplicated(keep= 'last')]
clientes_dic = clientes.set_index('NUM TDA').to_dict(orient = 'index')
data_limpia.loc[:,'Estado'] = data_limpia['Store_Id'].apply(lambda x: clientes_dic.get(x,dict()).get('ESTADO',None))
data_limpia.loc[:,'Cadena'] = data_limpia['Store_Id'].apply(lambda x: clientes_dic.get(x,dict()).get('CADENA',None))

Hay 32 distintos en la tabla


In [7]:
# Ahora podemos saber en que estado tiene ventas cada proveedor.
cadena_estado = data_limpia[['Cadena','Estado']].drop_duplicates()
# print(cadena_estado['Cadena'].value_counts())
# print(cadena_estado.sort_values(by='Cadena'))

# Hay un estado donde el proveedor 1 no tiene ventas en el archivo.
sin_ventas1 = clientes[~clientes['ESTADO'].isin(data_limpia[data_limpia['Cadena']== 'proveedor_1']['Estado'])]
sin_ventas1[sin_ventas1['CADENA'] == 'proveedor_1']['ESTADO'].unique()
# El estado es 'Alfthrilmad'

array(['Alfthrilmad'], dtype=object)

In [8]:
# Uno de los puntos es comprobar que los datos deben contener ventas para cada producto
# SKU es lo que identifica a las ventas con los productos entonces si  cada producto tiene ventas 
productos['Sku'].isin(data['Sku']).all() # Deberia ser Cierto

# Esto es falso, de manera especifica todas las ventas correspenden a 47 Sku distintos la tabla con sus 
# especificaciones:
sku = productos[productos['Sku'].isin(data['Sku'])]
# Todas pertenecen a las subcategorias 'Cereal bars' y 'Cereal box'.

# print(productos['Sku'].duplicated().sum() == 0) # nos dice que no hay 'Sku' repetidos 
# se genera un diccionario para poder agregar columnas de manera rapida
productos_dic = productos.set_index('Sku').to_dict(orient = 'index')

# Agrego la descripción del producto solo por que me gusta más ver los nombres.
data_limpia.loc[:,'Descripcion'] = data_limpia['Sku'].apply(lambda x: productos_dic[x]['Sku_Description'])

# Agregado simple de cada producto 
resumen_producto = data_limpia.groupby('Descripcion').agg({'Price':'sum','Pieces':'sum','Store_Id':'nunique',})
# print(resumen_producto)

In [9]:
# Quiero saber cuantos productos tienen ventas todos los días pues un requerimiento es tener ventas diarias 
# por producto. 
# Primero obtengo el numero total de días 
primero,ultimo = data_limpia['Date'].min(), data_limpia['Date'].max()
dias = ultimo-primero

# Ahora quiero saber cuantos días de venta tiene cada producto
dias_producto = data_limpia.groupby('Descripcion').agg({'Date':['min','max']}).apply(lambda x: x[1]- x[0],axis = 1).apply(lambda x: x.days)

# y quiero saber cuantos tienen 520 días
# print(str((días_producto == 520).sum())+ ' productos tienen ventas todos los días')
# print(dias_producto.sort_values(ascending=False)) 

17 productos tienen ventas todos los dias
Descripcion
Toast Mango Peach                             520
Grape-Nuts Mango Peach                        520
Crunch Mango Peach                            520
Frosted Chocolate Choc Chip                   520
Crunch Chocolate Choc Chip                    520
Chocos Mango Peach                            520
Grain Chocolate                               520
Crispy Rice Mango Peach                       520
Crisp Mango Peach                             520
Crunchy Bran Mango Peach                      520
Lucjy Charms Chocolate                        520
Muesli Chocolate                              520
Corn Chocolate Choc Chip                      520
Puffs Mango Peach                             520
Stars Mango Peach                             520
Sugar Chocolate Choc Chip                     520
Cups Mango Peach                              520
Fruit & Bran Mango Peach                      518
Coco Poops Mango Peach                        

## Resultados y conclusiones.

A primera vista los archivos tienen algunas inconsistencias en los formatos, eso en general hace que primero se tenga que hacer un poco de limpieza y algunos puntos son importantes preguntarlas al cliente para estar seguro de que los datos estén siendo interpretados de manera correcta.

Algunos problemas de formato son:
- Diferentes formatos para las fechas.
- Símbolo de pesos en la columna 'Price'.
- "SO Diaria Piezas" junto a la fecha.

En cuanto a problemas y dudas con los datos surgen las siguientes dudas:
- 580,000 de filas del archivo de ventas son filas con 0 en piezas y 0 en precio, eso es posiblemente un problema.
- 110 filas de ventas son filas con precio 0 y piezas mayor a 0, tal vez sea algun tipo de promociones pero posiblemente un error.
- La mitad de las filas tienen un valor con decimales en la columna de "Pieces", no necesariamente un error.

### Las principales preguntas.

Tratamos cada uno de los tres puntos requeridos por separado:
1. Estos datos deben tener ventas para todos los productos del catálogo de productos.
_Como lo vimos el archivo de venta solo contiene datos de 47 productos diferentes._
2. Estos datos deben cubrir las ventas en todo el país de dos grupos de supermercados.
_Tenemos que el proveedor 2 tiene tiendas en 32 estados y el proveedor 1 las tiene en 26. En cuanto a las ventas 
el proveedor 2 tiene ventas en cada uno de los estados y el proveedor 1 tiene ventas en 25 de 26 estados siendo la excepción 'Alfthrilmad'.
3. Estos datos deben tener ventas diarias por producto.
_De los 47 productos en el archivo de ventas 17 de ellos tienen registradas ventas en los 520 días, el numero numero de días que un producto tuvo ventas es 79._

### Conclusión.

 - ¿Tenemos lo que pedimos? 
_No, hay una gran cantidad de productos en el catalogo que no tienen ventas, también hay algunas ventas que no tienen asignada una tienda. También es necesario el preguntar por la columna Pieces._
 - ¿Nos falta algo?
_Si, faltan datos de ventas en algunas estados del proveedor 1 y por supuesto datos de venta de 2401 de un total de 2448_.

Creo que en el tema del formato lo más fácil es explicarle que un formato más estandarizado evita muchas errores que pueden más tardado un análisis, aunque el formato no es tan complicado de corregir.

Con respecto a el contenido en si de los datos, creo que preguntarle por la columna pieces y por las columnas con son 0 es muy util por que evitamos algunos errores que pueden afectar los resultados si no estamos seguros de como interpretarlos.